<h2 style="color:fuchsia;font-size:55px;font-family:Space mono;text-align:center;"><strong>Time Series Analysis 📈📈  </strong></h2> 


In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# **IMPORTING ESSENTIALS**

In [ ]:
!pip install pmdarima

In [ ]:
import warnings; warnings.filterwarnings('ignore')


import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pylab
from pylab import rcParams


plt.style.use('dark_background')
plt.rcParams['figure.figsize'] = 18,8
pd.set_option('display.max_columns', None)


from sklearn.metrics import mean_squared_error


from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.tsa.stattools import adfuller
from statsmodels.tsa.holtwinters import ExponentialSmoothing
from statsmodels.tsa.arima_model import ARIMA
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
import pmdarima as pm



# **READ AND MERGE THE DATA INTO A SINGLE DATAFRAME**

In [ ]:
df_holi = pd.read_csv('../input/store-sales-time-series-forecasting/holidays_events.csv')
df_oil = pd.read_csv('../input/store-sales-time-series-forecasting/oil.csv')
df_stores = pd.read_csv('../input/store-sales-time-series-forecasting/stores.csv')
df_trans = pd.read_csv('../input/store-sales-time-series-forecasting/transactions.csv')

df_train = pd.read_csv('../input/store-sales-time-series-forecasting/train.csv')
df_test = pd.read_csv('../input/store-sales-time-series-forecasting/test.csv')

In [ ]:
df_train1 = df_train.merge(df_holi, on = 'date', how='left')
df_train1 = df_train1.merge(df_oil, on = 'date', how='left')
df_train1 = df_train1.merge(df_stores, on = 'store_nbr', how='left')
df_train1 = df_train1.merge(df_trans, on = ['date', 'store_nbr'], how='left')
df_train1 = df_train1.rename(columns = {"type_x" : "holiday_type", "type_y" : "store_type"})

df_train1['date'] = pd.to_datetime(df_train1['date'])
df_train1['year'] = df_train1['date'].dt.year
df_train1['month'] = df_train1['date'].dt.month
df_train1['week'] = df_train1['date'].dt.isocalendar().week
df_train1['quarter'] = df_train1['date'].dt.quarter
df_train1['day_of_week'] = df_train1['date'].dt.day_name()

In [ ]:
df_train1.head(2)

# EDA

In [ ]:
df = df_train1 #copying the original df into a temp df which we can work on,just incase we dont mess up with the original one

In [ ]:
print(f'The merged dataframe has {df.shape[0]} rows and {df.shape[1]} columns')

In [ ]:
df.info()

In [ ]:
df.describe()

*Let's us check the time period of the data which is available*

In [ ]:
print(f'The data is available from {df.date.max()} to {df.date.min()}')

*for simplicity lets convert the date column values with only dates discarding the info about the hours, minutes and seconds*

In [ ]:
df['date'] = df['date'].dt.floor('d')


*For time-series analysis, let's only consider the 'data' and the 'sales' columns*

In [ ]:
df = df[['date', 'sales']]

In [ ]:
df = df.set_index(df.date)
df.drop('date', axis=1, inplace=True)


*Chesking if there are any missing values in our series, if there is any missing values then our time series analysis wont work*

In [ ]:
df.isna().sum()


*There are no missing values in our series, that means that the data is present for all of 2013 to 2017*
*The data is captured on daily basis, let's resample it to monthly basis to make analysis easier*

In [ ]:
df = df.resample('M').mean()

**NOTE :**
*The 'sales' column had many outliers, instead of treating them seperately, we can make use of 'median' while resampling instead of 'mean' to neutralize the effects of outliers,*
*Now the above series is an account of the almost average sales per month from 2013 to aug-2017*

# **VISUALIZE THE DATA**

In [ ]:
y = df['sales']
fig, ax = plt.subplots(figsize=(18, 8))
ax.plot(y,marker='o', markersize=8, linestyle='-', label='Monthly Mean Resample', color='fuchsia')
ax.set_ylabel('sales')
ax.set_title('Average sales per month')
ax.set_xlabel('years')
ax.grid(axis='x')
ax.legend();



*Looking at the figure above, by the eye-judgement we can see an upward trend in the avg sales and there seems to be some seasonality, but later on in this notebook we will verify these attributes*


# DECOMPOSE THE DATA

*By looking at the graph of sales data above, we can see a general increasing trend with no clear pattern of seasonal or cyclical changes. The next step is to decompose the data to view more of the complexity behind the linear visualization, once decomposed, we can clearly get :*
* *Observed (actual series)*
* *Trend*
* *Seasonality*
* *Residuals (irrgularities)*

In [ ]:
df_dec=seasonal_decompose(df,model='additive', extrapolate_trend='freq')
df_dec.plot();

*Looking at the four components above, we can say that the our avg sales per month has an upward trend overall and there is seasonality throughout, based on these factors, we can use suitable time-series forecasting model later*

# **CHECK FOR STATIONARITY**

*For the next step we have to check for the stationarity of the trend, a series is said to be stationary when*
* ***The mean doesnt change over time***
* ***The variance of the series doesnt change over time***

*and we have to make our data stationary if it is not stationary, because the time-series forecasting models make use of the assumption that there is no change in mean or variance in the data*

*The most two common methods used to check for stationarity are*
* ***Visualization***
* ***Augmented Dickey-Fuller test (ADF test)*** 

1. **Visualization**

In [ ]:
sales_mean = df.sales.rolling(window=12).mean()
sales_std = df.sales.rolling(window=12).std()

In [ ]:
fig, ax = plt.subplots(figsize=(18, 8))
ax.plot(df.sales,marker='x', markersize=8, label='Sales',color='cyan')
ax.plot(sales_mean,marker='o', markersize=4, label='rolling mean', color='red');
ax.plot(sales_std,marker='*', markersize=4, label='rolling std', color='royalblue');
ax.legend()
plt.xlabel('Years')
plt.ylabel('Avg sales')
plt.title('Check of stationarity')
ax.grid(axis='x')

*In the above figure, we are smoothening the series to remove the abrupt changes of the original data which can be helpful to judge the smoothened mean and std line of the data.*
*We can see that the mean of the data is not same through out but where as the standard deviation is almost constant.*

*We cannot confidently conclude that the data is stationary just by seeing the above insights, lets use adf test to check for the stationarity of the data next*

2. **Augmented Dickey-Fuller test (ADF test)**

*The augmented Dickey-Fuller test is a statistical test which is used to check for the stationarity of a time series, this test gives a p-value, test-statistic value and the critical values with confidence intervals stating the stationarity and based on the p-value obtained by the adf test we can reject or accpet the null hypothesis.*
* **H<sub>0</sub>  : 'The time series is not stationary'**
* **H<sub>a</sub>  : 'The time series is stationary'**

In [ ]:
dftest = adfuller(df.dropna(), autolag='AIC')
print('Test statistic = {:.3f}'.format(dftest[0]))
print('P-value = {:.3f}'.format(dftest[1]))
print('Critical values :')
for k, v in dftest[4].items():
    print('\t{}: {} - The data is {} stationary with {}% confidence'.format(k, v, 'not' if v<dftest[0] else '', 100-int(k[:-1])))

*As expected from the figure and also tested by the adf test for stationarity, we can confidently say that the original data is not stationary*

# **MAKING THE DATA STATIONARY**

*Now that we know about the stationarity of the data, we have to make it stationary, there are many ways to make a series stationary viz.*
* **Differencing**
* **Rolling window**
* **Transformation**

1. **DIFFERENCING**

*Differencing is a method of inducing a lag in the data which will remove the seasonal or cyclical patterns from the data*

In [ ]:
df_lag12 = df.sales - df.sales.shift(12)

In [ ]:
df_lag12.tail(5)

In [ ]:
y = df_lag12
fig, ax = plt.subplots(figsize=(18, 8))
ax.plot(y,marker='o', markersize=8, linestyle='-', label='Monthly Mean Resample', color='fuchsia')
ax.set_ylabel('sales')
ax.set_title('Average sales per month')
ax.set_xlabel('years')
ax.grid(axis='x')
ax.legend();

In [ ]:
dftest = adfuller(df_lag12.dropna(), autolag='AIC')
print('Test statistic = {:.3f}'.format(dftest[0]))
print('P-value = {:.3f}'.format(dftest[1]))
print('Critical values :')
for k, v in dftest[4].items():
    print('\t{}: {} - The data is {} stationary with {}% confidence'.format(k, v, 'not' if v<dftest[0] else '', 100-int(k[:-1])))

In [ ]:
df_dec_lag12 =seasonal_decompose(df_lag12.dropna(),model='additive', extrapolate_trend='freq')
df_dec.plot();

# **BUILDING A FORECASTING MODEL**

*Now that the data is stationary, we can go ahead and build a forecasting model*
*There are many forecasting models in time-series forecasting but depending on the data components we can pick the suitable model and build it*

* **Exponential Smoothing model is best for data without trend or seasonality**
* **Holt’s Method for data with a trend but no seasonality**
* **Holt-Winters for data with trend and/or seasonality**
* **ARIMA for data with trend and/or seasonality**


# **NAIVE MODEL IN TIME-SERIES FORECASTING**

In [ ]:
df['shifted_sales'] = df.sales.shift(1)

In [ ]:
df.head(4)

In [ ]:
fig, ax = plt.subplots(figsize=(18, 8))
ax.plot(df.sales,marker='*', markersize=8, label='Sales',color='white')
ax.plot(df.shifted_sales,marker='o', markersize=4, label='Shifted Sales', color='cyan');
plt.xlabel('Year')
plt.ylabel('avg sales')
plt.title('Naive model')
ax.legend()
plt.show()

In [ ]:
df = df.dropna()

In [ ]:
mse = mean_squared_error(df.sales, df.shifted_sales)

rmse = np.sqrt(mse)

print(f'The root mean square error of the naive model is : {rmse}')

*The naive forecasting model gave us a root mean square error of **57**, that means on an average a monthly sales prediction value might differ by a value of <u>+</u> ***57*** from the actual prediction*

# **HOLT-WINTER'S MODEL FOR FORECASTING**

In [ ]:
df = df.drop('shifted_sales',axis=1) # 'shifter_sales' not needed for holt-winters model

In [ ]:
hw = ExponentialSmoothing(df, seasonal_periods=12,trend='add', seasonal='add').fit()

In [ ]:
hw_preds = hw.forecast(steps=24)

In [ ]:
fig, ax = plt.subplots(figsize=(18, 8))
ax.plot(df,marker='*', markersize=8, label='Sales',color='fuchsia')
ax.plot(hw_preds,marker='*', markersize=8, label='Forecasted sales', color='orange');
plt.xlabel('Year')
plt.ylabel('avg sales')
plt.title('Holt-Winters model' )
ax.grid(axis='x')
ax.legend()
plt.show()

In [ ]:
hw_resid = hw.resid
hw_resid_sqr = np.square(hw_resid)
hw_resid_mse = hw_resid_sqr.sum()/len(hw_resid_sqr)
hw_resid_rmse = np.sqrt(hw_resid_mse)


print(f'The rmse value of Holt-Winters model is {hw_resid_rmse}')

*Comparing the rmse values of the Naive model and the holt-winters model, holt-winters model performs better with lesser rmse value, let's check with the final model  i.e **ARIMA** and compare how it performs other than these two*

# **ARIMA (Auto-Regressive Integrated Moving Average)**

*Now ARIMA takes in 3 components to work and hose are*
* Auto-Regressive (p)
* Integrated (differencing, d)
* Moving Average (q)

*And these values has to be found before giving it to the model, it can be done using **ACF** and **PACF** plots which will give the right values for the p and q values respectively*
   
*We can even use grid-seach technique to find the best values of p, q and d inorder to have least AIC value (AIC is Akike Information Criteria, it is a measure of the goodness of the time series model, lower the value of AIC better the model is)*


**PLOTTING ACF AND PACF PLOT**

In [ ]:
plot_acf(df);

*The shaded region is the critical value and the spikes is the correlation value between the 1st value and the next corresponding values, the spike which is in the shaded region is the optimal value of p*

In [ ]:
plot_pacf(df);

*The shaded region is the critical value and the spikes is the correlation value between the 1st value and the next corresponding values, the spike which is in the shaded region is the optimal value of q*

In [ ]:
dftrain = df.iloc[:38]
dftest = df.iloc[38:]

In [ ]:
arima_mod = ARIMA(dftrain, order=(5,1,3)) #p=5, d=1, q=3
arima_mod= arima_mod.fit(disp=0)
arima_preds = arima_mod.forecast(steps = 17)

In [ ]:
mse = mean_squared_error(dftest, arima_preds[0])
print(f'The root mean square error of ARIMA with (5,1,3) is {np.sqrt(mse)}')

In [ ]:
arima_preds = pd.Series(arima_preds[0], index=dftest.index)

In [ ]:
fig, ax = plt.subplots(figsize=(18, 8))
ax.plot(dftest,marker='*', markersize=8, label='Test sales', color='fuchsia');
ax.plot(arima_preds,marker='*', markersize=8, label='Arima_forecasting', color='grey');
plt.xlabel('Year')
plt.ylabel('avg sales')
plt.title('ARIMA' )
ax.grid(axis='x')
ax.legend()
plt.show()

*On comparing the Naie model, Holt-winter's model and ARIMA model, we can make out that the Holt-Winter's model performs well compared to all other models*

**HOLT-WINTER's FORECASTING MODEL FTW**